In [1]:
# netcdf/numpy/xray/stats
import xarray as xr

import sys
sys.path.append("/home/elilouis/sublimationofsnow/")
import sosutils
from metpy.units import units
import metpy
import pint_xarray
import altair as alt
alt.data_transformers.enable('json')
alt.renderers.enable('svg')
import pytz

import pandas as pd
import datetime as dt

from urllib.error import URLError
import os

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Inputs

In [2]:
sos_download_dir='/data2/elilouis/sublimationofsnow/sosnoqc'
DATE_FORMAT_STR = '%Y%m%d'
start_date = '20221201'; 
end_date = dt.datetime.strftime(dt.date.today() - dt.timedelta(days=1), DATE_FORMAT_STR)
PLANAR_FIT = True

# start_date = '20230219'
end_date = '20230110'



datelist = pd.date_range(
    dt.datetime.strptime(start_date, DATE_FORMAT_STR),
    dt.datetime.strptime(end_date, DATE_FORMAT_STR),
    freq='d'
).strftime(DATE_FORMAT_STR).tolist()

VARIABLE_NAMES = [
    # Sonic Anemometer Data for 4 towers
    'tc_1m_uw',     'spd_1m_uw',     'dir_1m_uw',     'u_1m_uw',   'v_1m_uw',   'w_1m_uw',   'u_u__1m_uw',    'v_v__1m_uw',    'w_w__1m_uw',    
        'u_w__1m_uw',    'v_w__1m_uw',  'u_tc__1m_uw',  'v_tc__1m_uw',   'u_h2o__1m_uw',  'v_h2o__1m_uw',   'w_tc__1m_uw',   'w_h2o__1m_uw',
    'tc_3m_uw',     'spd_3m_uw',     'dir_3m_uw',     'u_3m_uw',   'v_3m_uw',   'w_3m_uw',   'u_u__3m_uw',    'v_v__3m_uw',    'w_w__3m_uw',    
        'u_w__3m_uw',    'v_w__3m_uw',  'u_tc__3m_uw',  'v_tc__3m_uw',   'u_h2o__3m_uw',  'v_h2o__3m_uw',   'w_tc__3m_uw',   'w_h2o__3m_uw',
    'tc_10m_uw',    'spd_10m_uw',    'dir_10m_uw',    'u_10m_uw',  'v_10m_uw',  'w_10m_uw',  'u_u__10m_uw',   'v_v__10m_uw',   'w_w__10m_uw',   
        'u_w__10m_uw',   'v_w__10m_uw', 'u_tc__10m_uw', 'v_tc__10m_uw',  'u_h2o__10m_uw', 'v_h2o__10m_uw',  'w_tc__10m_uw',  'w_h2o__10m_uw',

    'tc_1m_ue',     'spd_1m_ue',     'dir_1m_ue',     'u_1m_ue',   'v_1m_ue',   'w_1m_ue',   'u_u__1m_ue',    'v_v__1m_ue',    'w_w__1m_ue',    
        'u_w__1m_ue',    'v_w__1m_ue',  'u_tc__1m_ue',  'v_tc__1m_ue',   'u_h2o__1m_ue',  'v_h2o__1m_ue',   'w_tc__1m_ue',   'w_h2o__1m_ue',
    'tc_3m_ue',     'spd_3m_ue',     'dir_3m_ue',     'u_3m_ue',   'v_3m_ue',   'w_3m_ue',   'u_u__3m_ue',    'v_v__3m_ue',    'w_w__3m_ue',    
        'u_w__3m_ue',    'v_w__3m_ue',  'u_tc__3m_ue',  'v_tc__3m_ue',   'u_h2o__3m_ue',  'v_h2o__3m_ue',   'w_tc__3m_ue',   'w_h2o__3m_ue',
    'tc_10m_ue',    'spd_10m_ue',    'dir_10m_ue',    'u_10m_ue',  'v_10m_ue',  'w_10m_ue',  'u_u__10m_ue',   'v_v__10m_ue',   'w_w__10m_ue',   
        'u_w__10m_ue',   'v_w__10m_ue', 'u_tc__10m_ue', 'v_tc__10m_ue',  'u_h2o__10m_ue', 'v_h2o__10m_ue',  'w_tc__10m_ue',  'w_h2o__10m_ue',

    'tc_1m_d',      'spd_1m_d',     'dir_1m_d',     'u_1m_d',   'v_1m_d',   'w_1m_d',   'u_u__1m_d',    'v_v__1m_d',    'w_w__1m_d',    
        'u_w__1m_d',    'v_w__1m_d',  'u_tc__1m_d',  'v_tc__1m_d',   'u_h2o__1m_d',  'v_h2o__1m_d',   'w_tc__1m_d',   'w_h2o__1m_d',
    'tc_3m_d',      'spd_3m_d',     'dir_3m_d',     'u_3m_d',   'v_3m_d',   'w_3m_d',   'u_u__3m_d',    'v_v__3m_d',    'w_w__3m_d',    
        'u_w__3m_d',    'v_w__3m_d',  'u_tc__3m_d',  'v_tc__3m_d',   'u_h2o__3m_d',  'v_h2o__3m_d',   'w_tc__3m_d',   'w_h2o__3m_d',
    'tc_10m_d',     'spd_10m_d',    'dir_10m_d',    'u_10m_d',  'v_10m_d',  'w_10m_d',  'u_u__10m_d',   'v_v__10m_d',   'w_w__10m_d',   
        'u_w__10m_d',   'v_w__10m_d', 'u_tc__10m_d', 'v_tc__10m_d',  'u_h2o__10m_d', 'v_h2o__10m_d',  'w_tc__10m_d',  'w_h2o__10m_d',

    'tc_2m_c',  'spd_2m_c',     'dir_2m_c',     'u_2m_c',   'v_2m_c',   'w_2m_c',   'u_u__2m_c',    'v_v__2m_c',    'w_w__2m_c',    
        'u_w__2m_c',    'v_w__2m_c',  'u_tc__2m_c',  'v_tc__2m_c',   'u_h2o__2m_c',  'v_h2o__2m_c',   'w_tc__2m_c',   'w_h2o__2m_c',
    'tc_3m_c',  'spd_3m_c',     'dir_3m_c',     'u_3m_c',   'v_3m_c',   'w_3m_c',   'u_u__3m_c',    'v_v__3m_c',    'w_w__3m_c',    
        'u_w__3m_c',    'v_w__3m_c',  'u_tc__3m_c',  'v_tc__3m_c',   'u_h2o__3m_c',  'v_h2o__3m_c',   'w_tc__3m_c',   'w_h2o__3m_c',
    'tc_5m_c',  'spd_5m_c',     'dir_5m_c',     'u_5m_c',   'v_5m_c',   'w_5m_c',   'u_u__5m_c',    'v_v__5m_c',    'w_w__5m_c',    
        'u_w__5m_c',    'v_w__5m_c',  'u_tc__5m_c',  'v_tc__5m_c',   'u_h2o__5m_c',  'v_h2o__5m_c',   'w_tc__5m_c',   'w_h2o__5m_c',
    'tc_10m_c', 'spd_10m_c',    'dir_10m_c',    'u_10m_c',  'v_10m_c',  'w_10m_c',  'u_u__10m_c',   'v_v__10m_c',   'w_w__10m_c',   
        'u_w__10m_c',   'v_w__10m_c', 'u_tc__10m_c', 'v_tc__10m_c',  'u_h2o__10m_c', 'v_h2o__10m_c',  'w_tc__10m_c',  'w_h2o__10m_c',
    'tc_15m_c', 'spd_15m_c',    'dir_15m_c',    'u_15m_c',  'v_15m_c',  'w_15m_c',  'u_u__15m_c',   'v_v__15m_c',   'w_w__15m_c',   
        'u_w__15m_c',   'v_w__15m_c', 'u_tc__15m_c', 'v_tc__15m_c',  'u_h2o__15m_c', 'v_h2o__15m_c',  'w_tc__15m_c',  'w_h2o__15m_c',
    'tc_20m_c', 'spd_20m_c',    'dir_20m_c',    'u_20m_c',  'v_20m_c',  'w_20m_c',  'u_u__20m_c',   'v_v__20m_c',   'w_w__20m_c',   
        'u_w__20m_c',   'v_w__20m_c', 'u_tc__20m_c', 'v_tc__20m_c',  'u_h2o__20m_c', 'v_h2o__20m_c',  'w_tc__20m_c',  'w_h2o__20m_c',

    
    # Temperature & Relative Humidity Array 
    'T_2m_c', 'T_3m_c', 'T_4m_c', 'T_5m_c', 'T_6m_c', 'T_7m_c', 'T_8m_c', 'T_9m_c', 'T_10m_c',
    'T_11m_c', 'T_12m_c', 'T_13m_c', 'T_14m_c', 'T_15m_c', 'T_16m_c', 'T_17m_c', 'T_18m_c', 'T_19m_c', 'T_20m_c',

    'RH_2m_c', 'RH_3m_c', 'RH_4m_c', 'RH_5m_c', 'RH_6m_c', 'RH_7m_c', 'RH_8m_c', 'RH_9m_c', 'RH_10m_c',
    'RH_11m_c','RH_12m_c','RH_13m_c','RH_14m_c','RH_15m_c','RH_16m_c','RH_17m_c','RH_18m_c','RH_19m_c','RH_20m_c',

    # Pressure Sensors
    'P_20m_c',
    'P_10m_c', 'P_10m_d', 'P_10m_uw', 'P_10m_ue',

    # Blowing snow/FlowCapt Sensors
    'SF_avg_1m_ue', 'SF_avg_2m_ue',

    # Apogee sensors
    "Vtherm_c", "Vtherm_d", "Vtherm_ue", "Vtherm_uw", 
    "Vpile_c", "Vpile_d", "Vpile_ue", "Vpile_uw",
    "IDir_c", "IDir_d", "IDir_ue", "IDir_uw",

    # Snow-level temperature arrays (towers D and UW)
    'Tsnow_0_4m_d', 'Tsnow_0_5m_d', 'Tsnow_0_6m_d', 'Tsnow_0_7m_d', 'Tsnow_0_8m_d', 'Tsnow_0_9m_d', 'Tsnow_1_0m_d', 'Tsnow_1_1m_d', 'Tsnow_1_2m_d', 'Tsnow_1_3m_d', 'Tsnow_1_4m_d', 'Tsnow_1_5m_d',
    'Tsnow_0_4m_uw', 'Tsnow_0_5m_uw', 'Tsnow_0_6m_uw', 'Tsnow_0_7m_uw', 'Tsnow_0_8m_uw', 'Tsnow_0_9m_uw', 'Tsnow_1_0m_uw', 'Tsnow_1_1m_uw', 'Tsnow_1_2m_uw', 'Tsnow_1_3m_uw', 'Tsnow_1_4m_uw', 'Tsnow_1_5m_uw',
    
    # Downward Facing Longwave Radiometer (tower D) - for measuring snow surface temperature
    'Rpile_out_9m_d',
    'Tcase_out_9m_d',
    
    # Upward facing shortwave radiometer (tower D) - for measuring incoming solar radiation!
    'Rsw_in_9m_d',
]

# Download SoS data

In [3]:
# We make sure that we aren't accessing variables that don't exist in the datasets
# This is necessary because some daily NetCDF files don't have all the expected variables
# (for example because an instrument was down). In that case, we want to add that variable
# to the dataset, filled with nans, which sosutils.merge_datasets_with_different_variables
# handles for us
datasets = []
datasets_safe = []
for date in datelist:
    try:
        ds = xr.open_dataset(sosutils.download_sos_data_day(date, sos_download_dir, cache=True, planar_fit=PLANAR_FIT))
    # Some dates are missing
    except URLError:
        print(f"failed on {date}, skipping")
    ds_new = ds[set(ds.data_vars).intersection(VARIABLE_NAMES)]
    datasets.append(ds_new)
    datasets_safe.append(ds_new)
sos_ds = sosutils.merge_datasets_with_different_variables(datasets, dim='time')

Caching...skipping download for 20221201
failed on 20221202, skipping
Caching...skipping download for 20221203
Caching...skipping download for 20221204
Caching...skipping download for 20221205
Caching...skipping download for 20221206
Caching...skipping download for 20221207
Caching...skipping download for 20221208
Caching...skipping download for 20221209
Caching...skipping download for 20221210
Caching...skipping download for 20221211
Caching...skipping download for 20221212
Caching...skipping download for 20221213
Caching...skipping download for 20221214
Caching...skipping download for 20221215
Caching...skipping download for 20221216
Caching...skipping download for 20221217
Caching...skipping download for 20221218
Caching...skipping download for 20221219
Caching...skipping download for 20221220
Caching...skipping download for 20221221
Caching...skipping download for 20221222
Caching...skipping download for 20221223
Caching...skipping download for 20221224
Caching...skipping download 

# Fill in missing timestamps (with nans)

Note that this occurs if there is missing data at the beginning or end of day - those timestamps will be left out of the dataset. 

In [4]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

def date_range(start_date, end_date, increment, period):
    result = []
    nxt = start_date
    delta = relativedelta(**{period:increment})
    while nxt <= end_date:
        result.append(nxt)
        nxt += delta

    return result


In [5]:
dt_list = date_range(pd.to_datetime(sos_ds.time.values[0]), pd.to_datetime(sos_ds.time.values[-1]), 5, 'minutes')

In [6]:
sos_ds = sos_ds.drop_duplicates(dim='time').reindex(time=dt_list)

# Calculate surface temperatures

Add new calculated variables to the dataset

From EOL (https://www.eol.ucar.edu/content/calculation-long-wave-radiation)
$$
R_{lw} = R_{pile} + SB * T_{case}^4
$$
And the steven-boltzman law
$$
T_{surface} = \Big( \frac {R_{lw}}{ \epsilon \sigma } \Big)^\frac{1}{4}
$$

In [7]:
SB = 5.67e-08 # steven boltzman constant, W/m^2/degK^4
SNOW_EMMISIVITY = 0.98
sos_ds['Tsurf_rad_d'] = ((sos_ds['Rpile_out_9m_d'] + SB * (sos_ds['Tcase_out_9m_d']+273.15)**4)/(SNOW_EMMISIVITY*SB))**(1/4) - 273.15

# Surface Temperature
# calculate from apogees
sos_ds['Tsurf_c'] = (['time'],  sosutils.apogee2temp(sos_ds, 'c').values)
sos_ds['Tsurf_d'] = (['time'],  sosutils.apogee2temp(sos_ds, 'd').values)
sos_ds['Tsurf_ue'] = (['time'],  sosutils.apogee2temp(sos_ds, 'ue').values)
sos_ds['Tsurf_uw'] = (['time'],  sosutils.apogee2temp(sos_ds, 'uw').values)

# Calculate potential and virtual potential temperatures in the air

Note that we use the `metpy` library here, which assigns units to each dataarray, in a "object oriented" way using the `pint` library. You don't really have to worry about this, only to know that it can make working with the data more confusing. Know that at the end of the following cell, when we assign the new variables to the dataset, we remove any remnants of the `pint` library, and we assign an attribute `units` to each new variable so that they somewhat match the variables provided in the original dataset.

In [8]:
# Potential Temperature
# iterate over pressure measurements
for i in range(2,21):
    absolute_temperature = sos_ds[f'T_{i}m_c'] * units.celsius
    relative_humidity = sos_ds[f'RH_{i}m_c']
    absolute_pressure = sos_ds['P_10m_c'] * units.millibar
    height_relative_to_10m_pressure_sensor = i*units.m - (10*units.m)

    height_adj_pressure = metpy.calc.add_height_to_pressure(
        absolute_pressure, 
        height_relative_to_10m_pressure_sensor
    )
    potential_temperature = metpy.calc.potential_temperature(    
            height_adj_pressure,
            absolute_temperature
    ).pint.to(units.celsius)
    mixing_ratio = xr.DataArray(relative_humidity/100) * metpy.calc.saturation_mixing_ratio(
        height_adj_pressure,
        absolute_temperature
    )
    air_density = metpy.calc.density(height_adj_pressure, absolute_temperature, mixing_ratio)
    virtual_potential_temperature = metpy.calc.virtual_temperature(
        potential_temperature,
        mixing_ratio,
    )

    virtual_temperature = metpy.calc.virtual_temperature(
        absolute_temperature,
        mixing_ratio,
    )
   
    sos_ds[f'Tpot_{i}m_c'] = (['time'], potential_temperature.pint.magnitude)
    sos_ds[f'Tpot_{i}m_c'] = sos_ds[f'Tpot_{i}m_c'].assign_attrs(units = str(potential_temperature.pint.units))

    sos_ds[f'Tvirtual_{i}m_c'] = (['time'], virtual_temperature.pint.magnitude)
    sos_ds[f'Tvirtual_{i}m_c'] = sos_ds[f'Tvirtual_{i}m_c'].assign_attrs(units = str(virtual_temperature.pint.units))

    sos_ds[f'Tpotvirtual_{i}m_c'] = (['time'], virtual_potential_temperature.pint.magnitude)
    sos_ds[f'Tpotvirtual_{i}m_c'] = sos_ds[f'Tpotvirtual_{i}m_c'].assign_attrs(units = str(virtual_potential_temperature.pint.units))

    sos_ds[f'airdensity_{i}m_c'] = (['time'], air_density.pint.magnitude)
    sos_ds[f'airdensity_{i}m_c'] = sos_ds[f'airdensity_{i}m_c'].assign_attrs(units = str(air_density.pint.units))

    sos_ds[f'mixingratio_{i}m_c'] = (['time'], mixing_ratio.pint.magnitude)
    sos_ds[f'mixingratio_{i}m_c'] = sos_ds[f'mixingratio_{i}m_c'].assign_attrs(units = str(mixing_ratio.pint.units))


# Calculate virtual potential temperatures at the snow surface

These are pretty much the same calculations as above, except we use the snow surface temperature from the longwave radiometer instead of air temperature.

Also, we assume that relative humidity at the snow surface is 100%

In [22]:
height_relative_to_10m_pressure_sensor = - (10*units.m)
absolute_temperature = sos_ds['Tsurf_rad_d']*units.celsius
relative_humidity = 100

potential_temperature = metpy.calc.potential_temperature(    
    height_adj_pressure,
    absolute_temperature
).pint.to(units.celsius)

mixing_ratio = xr.DataArray(relative_humidity/100) * metpy.calc.saturation_mixing_ratio(
        height_adj_pressure,
        absolute_temperature
    )
air_density = metpy.calc.density(height_adj_pressure, absolute_temperature, mixing_ratio)

virtual_potential_temperature = metpy.calc.virtual_temperature(
    potential_temperature,
    mixing_ratio,
)

virtual_temperature = metpy.calc.virtual_temperature(
    absolute_temperature,
    mixing_ratio,
)

sos_ds[f'Tsurfvirtual_rad_c'] = (['time'], virtual_temperature.pint.magnitude)
sos_ds[f'Tsurfvirtual_rad_c'] = sos_ds[f'Tsurfvirtual_rad_c'].assign_attrs(units = str(virtual_temperature.pint.units))

sos_ds[f'Tsurfpotvirtual_rad_c'] = (['time'], virtual_potential_temperature.pint.magnitude)
sos_ds[f'Tsurfpotvirtual_rad_c'] = sos_ds[f'Tsurfpotvirtual_rad_c'].assign_attrs(units = str(virtual_potential_temperature.pint.units))

sos_ds[f'Tsurfairdensity_rad_c'] = (['time'], air_density.pint.magnitude)
sos_ds[f'Tsurfairdensity_rad_c'] = sos_ds[f'Tsurfairdensity_rad_c'].assign_attrs(units = str(air_density.pint.units))

sos_ds[f'Tsurfmixingratio_rad_c'] = (['time'], mixing_ratio.pint.magnitude)
sos_ds[f'Tsurfmixingratio_rad_c'] = sos_ds[f'Tsurfmixingratio_rad_c'].assign_attrs(units = str(mixing_ratio.pint.units))

sos_ds[f'Tsurfpot_rad_c'] = (['time'], potential_temperature.pint.magnitude)
sos_ds[f'Tsurfpot_rad_c'] = sos_ds[f'Tsurfpot_rad_c'].assign_attrs(units = str(potential_temperature.pint.units))

# Calculate turbulent kinetic energy

In [23]:
sos_ds['tke_2m_c'] = 0.5*(sos_ds['u_u__2m_c'] + sos_ds['v_v__2m_c'] + sos_ds['w_w__2m_c'])
sos_ds['tke_3m_c'] = 0.5*(sos_ds['u_u__3m_c'] + sos_ds['v_v__3m_c'] + sos_ds['w_w__3m_c'])
sos_ds['tke_5m_c'] = 0.5*(sos_ds['u_u__5m_c'] + sos_ds['v_v__5m_c'] + sos_ds['w_w__5m_c'])
sos_ds['tke_10m_c'] = 0.5*(sos_ds['u_u__10m_c'] + sos_ds['v_v__10m_c'] + sos_ds['w_w__10m_c'])
sos_ds['tke_15m_c'] = 0.5*(sos_ds['u_u__15m_c'] + sos_ds['v_v__15m_c'] + sos_ds['w_w__15m_c'])
sos_ds['tke_20m_c'] = 0.5*(sos_ds['u_u__20m_c'] + sos_ds['v_v__20m_c'] + sos_ds['w_w__20m_c'])

sos_ds['tke_1m_uw'] = 0.5*(sos_ds['u_u__1m_uw'] + sos_ds['v_v__1m_uw'] + sos_ds['w_w__1m_uw'])
sos_ds['tke_3m_uw'] = 0.5*(sos_ds['u_u__3m_uw'] + sos_ds['v_v__3m_uw'] + sos_ds['w_w__3m_uw'])
sos_ds['tke_10m_uw'] = 0.5*(sos_ds['u_u__10m_uw'] + sos_ds['v_v__10m_uw'] + sos_ds['w_w__10m_uw'])

sos_ds['tke_1m_ue'] = 0.5*(sos_ds['u_u__1m_ue'] + sos_ds['v_v__1m_ue'] + sos_ds['w_w__1m_ue'])
sos_ds['tke_3m_ue'] = 0.5*(sos_ds['u_u__3m_ue'] + sos_ds['v_v__3m_ue'] + sos_ds['w_w__3m_ue'])
sos_ds['tke_10m_ue'] = 0.5*(sos_ds['u_u__10m_ue'] + sos_ds['v_v__10m_ue'] + sos_ds['w_w__10m_ue'])

sos_ds['tke_1m_d'] = 0.5*(sos_ds['u_u__1m_d'] + sos_ds['v_v__1m_d'] + sos_ds['w_w__1m_d'])
sos_ds['tke_3m_d'] = 0.5*(sos_ds['u_u__3m_d'] + sos_ds['v_v__3m_d'] + sos_ds['w_w__3m_d'])
sos_ds['tke_10m_d'] = 0.5*(sos_ds['u_u__10m_d'] + sos_ds['v_v__10m_d'] + sos_ds['w_w__10m_d'])

# Calculate gradients

The bouyancy term is the gradient in virtual potential temperature multiplied by gravity and divided by **absolute** virtual temperature. We calculate the gradient between the 2m sonic and the snow surface. Note that we use the **absolute** virtual temperature measured at the 2m instruments because this is a better estimate of average air temperature than the snow surface temperature (the temperature gradient is much steeper next to the snow surface than far from the snow surface). Also note that i've been bolding **absolute** virtual temperature because this value needs to be in units of Kelvin for this calculation.

The flow shear term is the gradient in wind speed. We calculate it over the distance between the 2m sonic and the snow surface, where wind speed is 0. 

We know that measurements at 2m are not always 2m above the snow surface, but we just estimate for now.

In [24]:
sos_ds.to_netcdf("example_dataset.nc")

# Smooth vertical latent heat flux data

In [ ]:
plt.figure(figsize=(3,2))
sns.histplot(sos_ds['w_h2o__1m_uw'][np.abs(sos_ds['w_h2o__1m_uw']) < .01], stat='density')
plt.xticks([-0.008, -0.004, 0, 0.004, 0.008], fontsize=8)
plt.yticks([])
plt.title("Latent heat flux measurements\nat one sonic (1m, UW tower)")

In [ ]:
def mad(data):
    return np.mean(np.absolute(data - np.mean(data)))

In [ ]:
sos_ds['w_h2o__1m_uw'] = sos_ds['w_h2o__1m_uw'].where(np.absolute(sos_ds['w_h2o__1m_uw']) < 4*mad(sos_ds['w_h2o__1m_uw']))
sos_ds['w_h2o__3m_uw'] = sos_ds['w_h2o__3m_uw'].where(np.absolute(sos_ds['w_h2o__3m_uw']) < 4*mad(sos_ds['w_h2o__3m_uw']))
sos_ds['w_h2o__10m_uw'] = sos_ds['w_h2o__10m_uw'].where(np.absolute(sos_ds['w_h2o__10m_uw']) < 4*mad(sos_ds['w_h2o__10m_uw']))
sos_ds['w_h2o__1m_ue'] = sos_ds['w_h2o__1m_ue'].where(np.absolute(sos_ds['w_h2o__1m_ue']) < 4*mad(sos_ds['w_h2o__1m_ue']))
sos_ds['w_h2o__3m_ue'] = sos_ds['w_h2o__3m_ue'].where(np.absolute(sos_ds['w_h2o__3m_ue']) < 4*mad(sos_ds['w_h2o__3m_ue']))
sos_ds['w_h2o__10m_ue'] = sos_ds['w_h2o__10m_ue'].where(np.absolute(sos_ds['w_h2o__10m_ue']) < 4*mad(sos_ds['w_h2o__10m_ue']))
sos_ds['w_h2o__1m_d'] = sos_ds['w_h2o__1m_d'].where(np.absolute(sos_ds['w_h2o__1m_d']) < 4*mad(sos_ds['w_h2o__1m_d']))
sos_ds['w_h2o__3m_d'] = sos_ds['w_h2o__3m_d'].where(np.absolute(sos_ds['w_h2o__3m_d']) < 4*mad(sos_ds['w_h2o__3m_d']))
sos_ds['w_h2o__10m_d'] = sos_ds['w_h2o__10m_d'].where(np.absolute(sos_ds['w_h2o__10m_d']) < 4*mad(sos_ds['w_h2o__10m_d']))
sos_ds['w_h2o__2m_c'] = sos_ds['w_h2o__2m_c'].where(np.absolute(sos_ds['w_h2o__2m_c']) < 4*mad(sos_ds['w_h2o__2m_c']))
sos_ds['w_h2o__3m_c'] = sos_ds['w_h2o__3m_c'].where(np.absolute(sos_ds['w_h2o__3m_c']) < 4*mad(sos_ds['w_h2o__3m_c']))
sos_ds['w_h2o__5m_c'] = sos_ds['w_h2o__5m_c'].where(np.absolute(sos_ds['w_h2o__5m_c']) < 4*mad(sos_ds['w_h2o__5m_c']))
sos_ds['w_h2o__10m_c'] = sos_ds['w_h2o__10m_c'].where(np.absolute(sos_ds['w_h2o__10m_c']) < 4*mad(sos_ds['w_h2o__10m_c']))
sos_ds['w_h2o__15m_c'] = sos_ds['w_h2o__15m_c'].where(np.absolute(sos_ds['w_h2o__15m_c']) < 4*mad(sos_ds['w_h2o__15m_c']))
sos_ds['w_h2o__20m_c'] = sos_ds['w_h2o__20m_c'].where(np.absolute(sos_ds['w_h2o__20m_c']) < 4*mad(sos_ds['w_h2o__20m_c']))

# Get Tidy Dataset

In [ ]:
tidy_df = sosutils.get_tidy_dataset(sos_ds, list(sos_ds.data_vars))

In [ ]:
tidy_df = sosutils.modify_df_timezone(tidy_df, pytz.UTC, pytz.timezone('US/Mountain'))

In [ ]:
tidy_df[tidy_df.measurement.apply(lambda x: x is None)].variable.unique()

# Calculate Ri and heat fluxes with turbpy

In [ ]:
SNOW_DEPTH = 0.9
PRESSURE_HEIGHT = 10
stab_titles, stab_methods, stab_dict = sosutils.get_turbpy_schemes()

In [ ]:
tidy_df_30Min = pd.DataFrame(
    tidy_df.set_index('time').groupby(
        ['measurement', 'variable', 'height',  'tower']
    )['value'].resample('30Min').mean()
).reset_index()

### HELP
Maybe its better to do these calculations with the original xarray dataset (like the calculations above). This would require that I explicitly declare all the variables used for modeled-flux calculations for each height/tower location which is probably better than the opaque things happening in sosutils.tidy_df_calculate_richardson_number_with_turbpy and sosutils.tidy_df_model_heat_fluxes_with_turbpy. Then I just need to come up with the new variable names and have those variable names parsed by the tidy_df functions.

In [ ]:
for tower in ['uw', 'ue', 'd', 'c']:
    print(f"Calculating modeled fluxes for tower {tower}")
    height_list = [2,3,5,10,20] if tower == 'c' else [1,3,10]
    for height in height_list:
        print(f"for height {height}")
        print("calculating results")
        RiBulk = sosutils.tidy_df_calculate_richardson_number_with_turbpy(
            tidy_df_30Min,
            tower = tower,
            height = height,    
            snowDepth = SNOW_DEPTH,
            pressure_height = PRESSURE_HEIGHT
        )

        # run models and get results
        (
            stability_correction,
            conductance_sensible,
            conductance_latent,
            sensible_heat,
            latent_heat,
            zeta
        ) = sosutils.tidy_df_model_heat_fluxes_with_turbpy(
            tidy_df_30Min,
            stab_titles, 
            stab_methods,
            stab_dict,
            tower, 
            height, 
            SNOW_DEPTH,
            PRESSURE_HEIGHT
        )

        print("adding richardson number to dataset")
        # combine results into tidy_df and calculate:
        # richardson number
        tidy_df_30Min = sosutils.tidy_df_add_variable(
            tidy_df_30Min,
            RiBulk,
            f'Ri_{height}m_{tower}',
            'Richardson Number',
            height,
            tower
        )

        print("adding modeled fluxes to dataset")
        # heat fluxes and the following calculations:
        # *  vertical water vapor moisture flux (m/s * g/m^2) using the latent heat of sublimation
        # Note that we use the opposite convention of turbpy - latent and sensible heat fluxes should be positive upward
        # therefore there are negative signs below
        for stab in stab_titles:
            tidy_df_30Min = sosutils.tidy_df_add_variable(
                tidy_df_30Min,
                - latent_heat[stab],
                f'LH_modeled_{stab}_{height}m_{tower}',
                f'latent heat flux modeled {stab}',
                height,
                tower,
            )
            tidy_df_30Min = sosutils.tidy_df_add_variable(
                tidy_df_30Min,
                - latent_heat[stab]/(2838),
                f'w_h20_modeled_{stab}_{height}m_{tower}',
                f'w_h2o_ modeled {stab}',
                height,
                tower,
            )
            tidy_df_30Min = sosutils.tidy_df_add_variable(
                tidy_df_30Min,
                - latent_heat[stab]/(2838),
                f'SH_modeled_{stab}_{height}m_{tower}',
                f'sensible heat flux modeled {stab}',
                height,
                tower,
            )

## Calculate RI and modeled fluxes using the fancy radiometer

In [ ]:
# get turby schemes
stab_titles, stab_methods, stab_dict = sosutils.get_turbpy_schemes()

SURFACE_TEMP_COL = 'Tsurf_rad_d'

for tower in ['uw', 'ue', 'd', 'c']:
#     print(f"Calculating modeled fluxes for tower {tower}")
#     height_list = [2,3,5,10,20] if
    print(f"Calculating modeled fluxes for tower {tower}")
    height_list = [2,3,5,10,20] if tower == 'c' else [1,3,10]
    for height in height_list:
        print(f"for height {height}")
        
        RiBulk = sosutils.tidy_df_calculate_richardson_number_with_turbpy(
            tidy_df_30Min,
            tower = tower,
            height = height,
            snowDepth = 1,
            pressure_height = 10,
            fillna_method = 'ffill',
            surface_temp_col_substitute = SURFACE_TEMP_COL
        )

        tidy_df_30Min = sosutils.tidy_df_add_variable(
            tidy_df_30Min,
            RiBulk,
            f'Ri_{height}m_{tower}_{SURFACE_TEMP_COL}',
            f'Richardson Number using {SURFACE_TEMP_COL}',
            height,
            tower
        )


        # run models and get results
        (
            stability_correction,
            conductance_sensible,
            conductance_latent,
            sensible_heat,
            latent_heat,
            zeta
        ) = sosutils.tidy_df_model_heat_fluxes_with_turbpy(
            tidy_df_30Min,
            stab_titles, 
            stab_methods,
            stab_dict,
            tower, 
            height, 
            1,
            10, 
            fillna_method='ffill',
            surface_temp_col_substitute = SURFACE_TEMP_COL
        )

        # heat fluxes and the following calculations:
        # *  vertical water vapor moisture flux (m/s * g/m^2) using the latent heat of sublimation
        # Note that we use the opposite convention of turbpy - latent and sensible heat fluxes should be positive upward
        # therefore there are negative signs below
        for stab in stab_titles:
            tidy_df_30Min = sosutils.tidy_df_add_variable(
                tidy_df_30Min,
                - latent_heat[stab],
                f'LH_modeled_{stab}_{height}m_{tower}_{SURFACE_TEMP_COL}',
                f'latent heat flux modeled {stab} using {SURFACE_TEMP_COL}',
                height,
                tower
            )
            tidy_df_30Min = sosutils.tidy_df_add_variable(
                tidy_df_30Min,
                - latent_heat[stab]/(2838),
                f'w_h20_modeled_{stab}_{height}m_{tower}_{SURFACE_TEMP_COL}',
                f'w_h2o_ modeled {stab} using {SURFACE_TEMP_COL}',
                height,
                tower
            )
            tidy_df_30Min = sosutils.tidy_df_add_variable(
                tidy_df_30Min,
                - latent_heat[stab]/(2838),
                f'SH_modeled_{stab}_{height}m_{tower}_{SURFACE_TEMP_COL}',
                'sensible heat flux modeled {stab} using {SURFACE_TEMP_COL}',
                height,
                tower
            )

## Calculate sublimation cumulative

In [ ]:
tidy_df_30Min = sosutils.tidy_df_add_variable(
    tidy_df_30Min,
    np.cumsum(tidy_df_30Min.query("variable == 'w_h2o__2m_c'")['value']*60*30).values*1000/(1e6),
    "cumulative sublimation (mm)",
    "cumulative sublimation (mm)",
    2,
    'c'
)

tidy_df_30Min = sosutils.tidy_df_add_variable(
    tidy_df_30Min,
    np.cumsum(tidy_df_30Min.query("variable == 'w_h2o__3m_c'")['value']*60*30).values*1000/(1e6),
    "cumulative sublimation (mm)",
    "cumulative sublimation (mm)",
    3,
    'c'
)

tidy_df_30Min = sosutils.tidy_df_add_variable(
    tidy_df_30Min,
    np.cumsum(tidy_df_30Min.query("variable == 'w_h2o__5m_c'")['value']*60*30).values*1000/(1e6),
    "cumulative sublimation (mm)",
    "cumulative sublimation (mm)",
    5,
    'c'
)

tidy_df_30Min = sosutils.tidy_df_add_variable(
    tidy_df_30Min,
    np.cumsum(tidy_df_30Min.query("variable == 'w_h2o__10m_c'")['value']*60*30).values*1000/(1e6),
    "cumulative sublimation (mm)",
    "cumulative sublimation (mm)",
    10,
    'c'
)

tidy_df_30Min = sosutils.tidy_df_add_variable(
    tidy_df_30Min,
    np.cumsum(tidy_df_30Min.query("variable == 'w_h2o__20m_c'")['value']*60*30).values*1000/(1e6),
    "cumulative sublimation (mm)",
    "cumulative sublimation (mm)",
    20,
    'c'
)

## Cache downloaded and processed 

In [ ]:
if PLANAR_FIT:
    tidy_df_30Min.to_csv(f'tidy_df_30Min_{start_date}_{end_date}_planar_fit.csv', index=False)
else:
    tidy_df_30Min.to_csv(f'tidy_df_30Min_{start_date}_{end_date}_noplanar_fit.csv', index=False)

# Load disdrometer data and calculate "days since precip"

In [ ]:
import act

In [ ]:
files = act.discovery.download_data(
    os.getenv("ARM_USERNAME"),
    os.getenv("ARM_TOKEN"),
    'gucldM1.b1',
    start_date,
    end_date,
    output='/data2/elilouis/sublimationofsnow/gucldM1.b1'
)

In [ ]:
disdro_ds = xr.open_mfdataset(files)
disdro_df = disdro_ds.to_dataframe().reset_index()

In [ ]:
disdro_daily_max_precip_date_df = disdro_df.set_index(
    "time"
)[['precip_rate']].resample(
    "1440Min"
).max().reset_index()

disdro_daily_mean_precip_date_df = disdro_df.set_index(
    "time"
)[['precip_rate']].resample(
    "1440Min"
).max().reset_index()

s = disdro_daily_max_precip_date_df.groupby(disdro_daily_max_precip_date_df['precip_rate'].ne(0).cumsum())['time'].transform('first')
disdro_daily_max_precip_date_df['days_since_precip'] = (disdro_daily_max_precip_date_df['time'] - s).dt.days

In [ ]:
daily_mean_precip_df = disdro_df.groupby("time")['precip_rate'].max()

daily_mean_precip_df = pd.DataFrame(daily_mean_precip_df.resample("1440Min").mean()*24/10) # resample to one day in cm/hr SWE

In [ ]:
disdro_daily_max_precip_date_df['daily_precip (cm)'] =  daily_mean_precip_df['precip_rate'].values

In [ ]:
disdro_daily_max_precip_date_df

## Cache Downloaded Disdrometer data

In [ ]:
disdro_daily_max_precip_date_df.to_csv("disdro_daily_max_precip_date_df.csv")